In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from dateutil.parser import parse
import time
import os
import pickle

In [2]:
# Anzahl der angezeigten Zeilen in JNP einstellen
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
# Pickl-Dateipfade vorbereiten:
data_path_weather = '/home/paul/python_projects/masterthesis/wma/data/wetter/'

pkl_file_wetter_step_1 = 'daten_wetter_step_1.p'

In [4]:
df_wetter_1 = pickle.load(open(data_path_weather + pkl_file_wetter_step_1, 'rb'))

In [5]:
df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01350,1,C,N","011265,1,N,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"210,1,N,0051,1","01200,1,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0051,1","01350,1,C,N","011265,1,N,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"230,1,N,0051,1","99999,9,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0067,1","01200,1,C,N","011265,1,N,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00
